In [1]:
from langchain.document_loaders import WikipediaLoader,TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore

C:\Users\PC\anaconda3\envs\langchain\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Cargar documentos

In [2]:
loader=WikipediaLoader(query='Daule',lang="es")
documents=loader.load()

C:\Users\PC\anaconda3\envs\langchain\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\PC\anaconda3\envs\langchain\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [3]:
len(documents)

24

# Split

In [4]:
text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs=text_splitter.split_documents(documents)

Created a chunk of size 530, which is longer than the specified 500
Created a chunk of size 523, which is longer than the specified 500
Created a chunk of size 554, which is longer than the specified 500
Created a chunk of size 746, which is longer than the specified 500
Created a chunk of size 1051, which is longer than the specified 500
Created a chunk of size 838, which is longer than the specified 500


In [5]:
len(docs)

56

# Conectar con embeddings de OpenAI

In [6]:
f=open('../openai_api_key.txt')
api_key=f.read()

funcion_embedding=OpenAIEmbeddings(openai_api_key=api_key)

# Incrustar documentos en BD Vectores

In [7]:
persist_path='../persistence/ejemplo_wiki_bd'

vector_store=SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=funcion_embedding,
    persist_path=persist_path,
    serializer="parquet"
)

In [8]:
vector_store.persist()

# 5a Consulta Normal

In [11]:
consulta="¿Por qué se conoce a Daule como 'Capital arrocera del Ecuador'?"
docs=vector_store.similarity_search(consulta)
print(docs[0].page_content)

Daule, también conocida como Santa Clara de Daule, es una ciudad ecuatoriana; cabecera del cantón homónimo y la tercera urbe más grande y poblada de la provincia de Guayas. Se localiza al centro-sur de la región litoral ecuatoriana, en una extensa llanura, en la orilla izquierda del río Daule, a una altitud de 7 m s. n. m. y con un clima tropical de sabana de 24,4 °C en promedio.
Es denominada como la "capital arrocera del Ecuador" por su importante producción de arroz. En el censo de 2022 tenía una población de 161.498 habitantes lo que la convierte en la décima tercera ciudad más poblada del país. Forma parte de la área metropolitana de Guayaquil, pues su actividad económica, social y comercial está fuertemente ligada a Guayaquil, siendo una ciudad dormitorio para miles de personas que se trasladan a Guayaquil por vía terrestre diariamente. El conglomerado alberga a 3.618.450 habitantes, y ocupa la primera posición entre las conurbaciones del Ecuador.
Sus orígenes datan del siglo XVI

# 5b Consulta con comprensión contextual usando LLMs

In [12]:
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [13]:
llm=ChatOpenAI(
    temperature=0,
    openai_api_key=api_key
)
compressor=LLMChainExtractor.from_llm(llm)

In [14]:
compression_retriever=ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever()
)

In [15]:
compressed_docs=compression_retriever.invoke(consulta)

In [16]:
compressed_docs[0].page_content

'Es denominada como la "capital arrocera del Ecuador" por su importante producción de arroz.'